# Trabajo práctico  1

## 1)
Métrica de performance (P): Poder mover todos los discos a cualquier otra barra distinta de la inicial, de forma tal en que los anillos estén ordenados de forma decreciente (comenzando desde la base).

Ambiente (E): El conjunto de barras y discos, y cada disco con su respectivo peso para ser odenado.

Actuadores (A): Mover anillos de una barra a otra de forma tal de que un anillo nunca se posicione sobre otro de menor peso.

Sensores (S): Observar las barras y la posición de los anillos en ellas.

## 2)
El entorno de trabajo es:
- Totalmente observable, ya que el agente puede saber en que posición está cada anillo en cada estado y la totalidad de las barras.
- Determinista, ya que no hay agentes externos que interfieran, los pesos de los anillos son estáticos, y la cantidad de barras no varía; en otras palabras, a partir de un estado y un movimiento, siempre se obtiene otro estado resultante, sin importar cuantas veces se realice el experimento.
- Secuencial, pues un estado depende del estado anterior y un movimiento.
- Discreto, ya que el espacio de estados se despliega de a un movimiento, y es finito.
- Estático, ya que no varían las restricciones, pesos de los anillos ni cantidad de barras durante la ejecución.
- Agente individual, ya que opera un único agente cuyo objetivo es resolver el problema pasando la torre completa a otra barra.
- Episódico, ya que cada ejecución es independiente de las demás.

## 3)
- Estado: cada estado representa una configuración de anillos en las barras, en donde las barras siempre serán fijas, pero la cantidad de anillos por barra varían en cada estado.
- Espacio de estados: por cada estado, se crean estados siguientes con movimientos válidos de anillos. Sean $d$ y $t$ la cantidad de discos y de barras, respectivamente, se generan como máximo $d(t-1)$ estados siguientes a partir de uno dado.
- Árbol de búsqueda: espacio de estados junto con costo de transición o penalización por pasar de un estado al siguiente.
- Nodo de búsqueda: Un estado con información útil, como por ejemplo penalización o costo por cada transición a un estado siguiente.
- Objetivo: trasladar todos los discos a una torre diferente de la inicial.
- Acción: mover un disco a la vez de forma válida (cada anillo podrá estar arriba de uno con mayor peso únicamente).
- Frontera: estados con configuraciones de anillos que se crearon a partir de uno anterior pero que aún no se evaluaron con respecto a la búsqueda del camino mínimo.


## 4)
Se optó por implementar el algoritmo A* junto a una serie de heurísticas y tipos de datos abstractos para modelar el problema en cuestión.

En principio, se optó por abstraer los componentes del puzzle en tipos de datos abstractos llamados Ring y Tower, siendo los anillos y barras, respectivamente. A priori, estos componentes no tienen lógica, sino que sirven como DTOs. El orden de los anillos se basa en su peso (`weight`), el cual se asigna en su creación.

**Importante:** el código fue implementado utilizando Python 3.12.

In [1]:
from dataclasses import dataclass, field
from typing import List
from typing import Dict, Optional
from queue import PriorityQueue

In [2]:
@dataclass
class Ring:
    weight: int

    def __eq__(self, other):
        if other.__class__ is self.__class__:
            return self.weight == other.weight
        return NotImplemented

    def __str__(self):
        return f"{self.weight}"
    
@dataclass
class Tower:
    rings: List[Ring]

    @classmethod
    def empty(cls) -> "Tower":
        return Tower([])

    def __eq__(self, other):
        if other.__class__ is self.__class__:
            return len(self.rings) == len(other.rings) and all(r1 == r2 for r1, r2 in zip(self.rings, other.rings))
        return NotImplemented

    def __str__(self):
        return "{" + ",".join(str(ring) for ring in self.rings) + "}"

Además, resultó conveniente definir la clase State, la cual representa un nodo válido del espacio de esatados, el cual, además de contener las estructuras internas para representar el problema, también consta de lógica para poder generar los siguientes estados válidos a partir de sí mismo. Es decir, dado un estado, genera todas las combinaciones de movimientos válidos de todos los anillos que puedan moverse.

In [3]:
@dataclass(order=True)
class State:
    priority: float
    accumulated_cost: float = field(compare=False)
    towers: List[Tower] = field(compare=False)
    parent: Optional["State"] = field(compare=False)

    @classmethod
    def with_towers(cls, towers: List[Tower]) -> "State":
        return State(priority=0, accumulated_cost=0, towers=towers, parent=None)

    def expand(self) -> List["State"]:
        """
        Genera todas las posibles combinaciones de estados válidos a partir del actual, teniendo en cuenta
        que cada disco sólo puede moverse por encima de otro de mayor peso. 
        """
        new_states = []
        for current_tower_index, current_tower in enumerate(self.towers):
            if len(current_tower.rings) == 0:
                continue

            movable_ring = current_tower.rings[0]
            for destination_tower_index, destination_tower in enumerate(self.towers):
                if destination_tower_index == current_tower_index:
                    continue

                if len(destination_tower.rings) == 0 or movable_ring.weight < destination_tower.rings[0].weight:
                    new_towers = [Tower(tower.rings.copy()) for tower in self.towers]
                    new_towers[current_tower_index].rings.pop(0)
                    new_towers[destination_tower_index].rings.insert(0, movable_ring)
                    new_states.append(
                        State(
                            priority=0,
                            accumulated_cost=self.accumulated_cost + 1,
                            towers=new_towers,
                            parent=self))
        return new_states
    
    def __eq__(self, other):
        if other.__class__ is self.__class__:
            return self.towers == other.towers
        return NotImplemented

    def __str__(self):
        return " + ".join(str(tower) for tower in self.towers)

Siendo el algoritmo elegido el A*, el cual consiste en evaluar la suma de los costos de transición entre los nodos para obtener el camino más corto a una solución junto a la utilización de una heurística. En este caso, el costo de un nodo a otro se computó sumando el mismo valor para todos (1), ya que solo importa el conteo de la cantidad de estados intermedios hasta llegar a la solución. Es por ello, que al no tener un costo asociado, se podría haber utilizado BFS, Dijkstra, A*, entre otros; cualquiera hubiese sido apto para este caso. La heurística se pasa por parámetro y consiste en una función arbitraria que aporta información en base al estado actual.

In [4]:
class AStar:
    
    def __init__(self, heuristic):
        self.heuristic = heuristic

    def execute(self, initial_state: State, goal: State) -> Optional[State]:
        counter = 0
        frontier: PriorityQueue[State] = PriorityQueue()
        visited_states: Dict[str, State] = {}

        frontier.put(initial_state)

        visited_states[str(initial_state)] = initial_state 
        
        while not frontier.empty():
            current_state = frontier.get()
            
            counter += 1
            
            if current_state == goal:
                print(f'Cantidad de pasos dados: {counter}')
                return current_state
        
            for next_state in current_state.expand():

                new_cost = current_state.accumulated_cost + 1

                visited_next_state = visited_states.get(str(next_state))

                if visited_next_state is None or visited_next_state.accumulated_cost > new_cost:
                    estimated_left_cost = self.heuristic(next_state, goal)

                    next_state.accumulated_cost = new_cost
                    next_state.priority = new_cost + estimated_left_cost

                    visited_states[str(next_state)] = next_state
                    frontier.put(next_state)

        return None

Se definieron varias heurísticas para evaluarlas individualmente y eventualmente combinar las mejores de ellas. Todas las heurísticas evalúan los estados siguientes del actual, ya que toda información pasada del estado actual será compartida por todos sus estados hijos (siguientes), lo cual no aporta demasiada información.

### Heurística 1: por coincidencia en torres
A partir de un estado actual, la heurística consiste en evaluar en que barra concentra más anillos uno de sus estados siguientes y los pondera en base a si dicha barra es la misma que la que se tiene en el estado final. Dicho de otra forma, penaliza a un estado siguiente si no concentra la mayoría de sus anillos en la misma barra objetivo. Para ello, se definió la función que devuelve el índice de la barra con la mayor cantidad de anillos. 



In [5]:
def tower_coincidence_heuristic(next_state: State, goal_state: State):    
    def most_populated_tower(towers: List[Tower]) -> int:
        indexed_rings_number = [len(tower.rings) for tower in towers]
        most_populated_tower = max(indexed_rings_number)
        return indexed_rings_number.index(most_populated_tower)
    
    if most_populated_tower(next_state.towers) == most_populated_tower(goal_state.towers):
        return -1
    return 1

### Heurística 2: por anillos a mover hasta el objetivo
Consiste en contar la cantidad de anillos en las torres que no coincidan con las torres obetivo; esa es la cantidad de anillos que habrá que hacer hasta llegar al objetivo. Mientras más falten mover, mayor será el costo del siguiente estado.

In [6]:
def rings_to_be_moved_heuristic(next_state: State, goal_state: State) -> int:
    goal_tower = next(tower for tower in goal_state.towers if tower.rings)
    rings_to_be_moved = 0
    
    for next_tower in next_state.towers:
        for ring in next_tower.rings:
            if ring not in goal_tower.rings:
                rings_to_be_moved += 1
                
    return rings_to_be_moved

### Heurística 2: por distancia manhattan entre posiciones de anillos
Se obtiene el índice de la posición de los anillos y se calcula la distancia entre la posición actual y la posición de la torre objetivo. Además, se tienen en cuenta los índices de las barras en las que están actualmente, contra el índice de la barra objetivo.

In [7]:
def manhattan_distance_heuristic(next_state: State, goal_state: State) -> int:
    goal_tower_index = next(i for i, tower in enumerate(goal_state.towers) if tower.rings) 
    goal_tower_rings = goal_state.towers[goal_tower_index].rings
    total_distance = 0
    
    for next_tower_index, next_tower in enumerate(next_state.towers):
        for next_ring_index, next_ring in enumerate(next_tower.rings):
            goal_ring_index = goal_tower_rings.index(next_ring)
            total_distance += abs(goal_tower_index - next_tower_index) + abs(goal_ring_index - next_tower_index)

    return total_distance


Se instancia el algoritmo de búsqueda A* tres veces con el mismo estado inicial pero con diferentes heurísticas. 

In [8]:
def run_algorithm(heuristic):
    initial_state_towers = [Tower([Ring(weight=w) for w in range(8)]), Tower([]), Tower([])]
    initial_state = State.with_towers(initial_state_towers)
    
    goal_state_towers = [Tower([]), Tower([]), Tower([Ring(weight=w) for w in range(8)])]
    goal = State.with_towers(goal_state_towers)
    
    search_algorithm = AStar(heuristic)
    result = search_algorithm.execute(initial_state, goal)
    print(f"Resultado: {result}")

run_algorithm(tower_coincidence_heuristic)
run_algorithm(rings_to_be_moved_heuristic)
run_algorithm(manhattan_distance_heuristic)

Cantidad de pasos dados: 6387
Resultado: {} + {} + {0,1,2,3,4,5,6,7}
Cantidad de pasos dados: 6467
Resultado: {} + {} + {0,1,2,3,4,5,6,7}
Cantidad de pasos dados: 6175
Resultado: {} + {} + {0,1,2,3,4,5,6,7}


Puede verse que la mejor heurística fue la distancia manhattan, seguido por la de coincidencia de torres, dejando atras a la heurística de movimientos necesarios. Por ello, probaremos la combinación de las dos mejores como heurística.

In [9]:
def combined(next_state: State, goal_state: State) -> int:
    return (tower_coincidence_heuristic(next_state, goal_state) +
            manhattan_distance_heuristic(next_state, goal_state))

run_algorithm(combined)

Cantidad de pasos dados: 6145
Resultado: {} + {} + {0,1,2,3,4,5,6,7}


Mejora el resultado por poco, lo cual lo hace despreciable frente al procesamiento adicional que supone frente a la utilización de la heurística basada en la distnaica manhattan sin otra combinación.

## 5
### Complejidad temporal
Sea $D$ el conjunto de discos de tamaño $d$ y $t$ la cantidad de barras del problema, tenemos que para cada estado, los anillos que puedan moverse (los que estén en el tope), pueden moverse a lo sumo de $(t-1)$ veces, es decior, a culquiera de las demás barras. Esto ocurre solamente cuando el anillo a mover es el de menor peso. Luego, cada anillo siguiente podrá moverse a cualquier otra torre excepto a aquellas que posean anillos con menor peso que el. Entonces, en general, definimos $M(D_i)$ como la cantidad de movimientos para cada anillo $D_i$, el cual viene dado por la cantidad de torres como $M(D_i)=t-i-1$, ya que podrá moverse en tanto no sea el más pesado $i=d$ y a medida que sea más liviano ($i=0$) tendrá más libertad de movimiento. Ahora bien, no todos los anillos pueden moverse, ya que sólo lo harán aquellos que se encuentren en el tope de cada barra; es decir, a lo sumo $t-1$ anillos podrán moverse por cada estado, ya que siempre hay uno más pesado que el resto que no podrá hacerlo. Entonces, por cada estado se general a lo sumo $(t-1)M(D_i)$ movimientos posibles. En el peor de los casos, M(D_i=0)=t-1, por lo que cada estado tendrá en el peor de los casos $(t-1)(t-1)$ movimientos. Como podemos tener un máximo de $t^d$ estados, tenemos que el peor caso viene dado por $O\left((t-1)(t-1){t^d}\right)$. No obstante, cada vez que se generan estados, se insertan en una cola de prioridad, lo cual conlleva algunos pasos adicionales; si por cada estado podemos generar $(t-1)(t-1)$ estados siguientes, y la complejidad de insertar un elemento en una cola de prioridad es de $log n$ con $n$ como la cantidad de elementos en la cola, en donde en el peor de los casos puede ser $t^d$, entonces la expresión se actualiza como. Por otro lado, dentro del mismo loop que la inserción y recuperación de elementos de la frontera, las heurísticas poseen una complejidad propia. En el caso de la combinada consistente en manhattan y los anillos restantes por mover, ambas iteran sobre torres y luego, de forma anidada, sobre anillos en cada torre. En el peor de los casos tenemos $t*d$ pasos por cada uno, pero como se ejecutan secuencialmente, nos quedamos con el máximo peor caso entre ellos (como son iguales, es $t*d$). Luego, nos quedamos con el máximo entre la complejidad de la heurística y la utilización de la cola. Luego, la complejidad temporal completa queda definida como $O\left (t^d(t-1)(t-1) max\left(log(t^d) + td\right)\right)=O\left (t^d(t-1)(t-1) td \right)$. Simplificando, tenemos $O(t^{d+1}(t-1)^2d)$.

### Complejidad espacial
Como hay $t^d$ estados posibles y cada barra puede tener todos los discos en el peor de los casos, tenemos que el peor caso de la complejidad espacial viene dada por $O\left(t^dd \right)$.

## 6
A continuación se listan las métricas de tiempo de ejecución y de memoria del algoritmo con la heurística combinada

In [10]:
!pip install memory_profiler
!pip install numpy 


[notice] A new release of pip is available: 23.2.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [11]:
%load_ext memory_profiler

In [12]:
%%timeit
run_algorithm(combined)

Cantidad de pasos dados: 6145
Resultado: {} + {} + {0,1,2,3,4,5,6,7}
Cantidad de pasos dados: 6145
Resultado: {} + {} + {0,1,2,3,4,5,6,7}
Cantidad de pasos dados: 6145
Resultado: {} + {} + {0,1,2,3,4,5,6,7}
Cantidad de pasos dados: 6145
Resultado: {} + {} + {0,1,2,3,4,5,6,7}
Cantidad de pasos dados: 6145
Resultado: {} + {} + {0,1,2,3,4,5,6,7}
Cantidad de pasos dados: 6145
Resultado: {} + {} + {0,1,2,3,4,5,6,7}
Cantidad de pasos dados: 6145
Resultado: {} + {} + {0,1,2,3,4,5,6,7}
Cantidad de pasos dados: 6145
Resultado: {} + {} + {0,1,2,3,4,5,6,7}
Cantidad de pasos dados: 6145
Resultado: {} + {} + {0,1,2,3,4,5,6,7}
Cantidad de pasos dados: 6145
Resultado: {} + {} + {0,1,2,3,4,5,6,7}
Cantidad de pasos dados: 6145
Resultado: {} + {} + {0,1,2,3,4,5,6,7}
Cantidad de pasos dados: 6145
Resultado: {} + {} + {0,1,2,3,4,5,6,7}
Cantidad de pasos dados: 6145
Resultado: {} + {} + {0,1,2,3,4,5,6,7}
Cantidad de pasos dados: 6145
Resultado: {} + {} + {0,1,2,3,4,5,6,7}
Cantidad de pasos dados: 6145
Resu

In [13]:
import numpy as np
from memory_profiler import memory_usage

def memory_wrapper():
    run_algorithm(combined)

runs = 7
mem_usages = []

for _ in range(runs):
   mem_usages.append(max(memory_usage(memory_wrapper, retval=False)))

mean = np.mean(mem_usages)
standard_deviation = np.std(mem_usages)

print(f'{mean:.6f} MiB ± {standard_deviation:.6f} MiB (mean ± std. dev. of {runs} runs)')

Cantidad de pasos dados: 6145
Resultado: {} + {} + {0,1,2,3,4,5,6,7}
Cantidad de pasos dados: 6145
Resultado: {} + {} + {0,1,2,3,4,5,6,7}
Cantidad de pasos dados: 6145
Resultado: {} + {} + {0,1,2,3,4,5,6,7}
Cantidad de pasos dados: 6145
Resultado: {} + {} + {0,1,2,3,4,5,6,7}
Cantidad de pasos dados: 6145
Resultado: {} + {} + {0,1,2,3,4,5,6,7}
Cantidad de pasos dados: 6145
Resultado: {} + {} + {0,1,2,3,4,5,6,7}
Cantidad de pasos dados: 6145
Resultado: {} + {} + {0,1,2,3,4,5,6,7}
Cantidad de pasos dados: 6145
Resultado: {} + {} + {0,1,2,3,4,5,6,7}
Cantidad de pasos dados: 6145
Resultado: {} + {} + {0,1,2,3,4,5,6,7}
Cantidad de pasos dados: 6145
Resultado: {} + {} + {0,1,2,3,4,5,6,7}
Cantidad de pasos dados: 6145
Resultado: {} + {} + {0,1,2,3,4,5,6,7}
Cantidad de pasos dados: 6145
Resultado: {} + {} + {0,1,2,3,4,5,6,7}
Cantidad de pasos dados: 6145
Resultado: {} + {} + {0,1,2,3,4,5,6,7}
Cantidad de pasos dados: 6145
Resultado: {} + {} + {0,1,2,3,4,5,6,7}
83.948661 MiB ± 0.007059 MiB (mean

Vemos entonces que el tiempo de ejecución promedio fue de 127ms con una desviación estándar de 1.15ms. En el caso de la memoria, la media fue de 95.00690 con una desviación estándar de 0.170317. No obstante, no son métricas certeras pues depende del estado del sistema operativo. 

# 7
Suponendo que tenemos $d=8$ anillos, y que el caso óptimo es $2^d-1=255$, y tomando el mejor resultado consistente en la heurística combinada, la cual resolvió el problema en $6145$, podemos ver que dista de la solución óptima por 5890 pasos. Como las salidas son estáticas, no calculé el promedio, sino que utilicé la salida directamente.